# Kernel => Restart & Run All
<hr>

### 랜덤으로 계획 생성하기(1번) => 이벤트의 개수만 지정
### 사용자가 계획 생성하기(2번) => 이벤트의 개수에 따라 시작날짜, 종료날짜, 얻을 수 있는 경험치, 합격 확률 직접 지정
<hr>

### 출력: 
    1. 기대 경험치 최대화
    2. 실제 경험치 최대화
    3. 이벤트 개수 최대화

In [ ]:
from datetime import datetime, timedelta
import random

In [ ]:
professor = ["임태수 교수님","김상근 교수님","한경수 교수님","최정열 교수님",
             "임상순 교수님", "강영명 교수님","심미나 교수님","박미옥 교수님",
             "고혜경 교수님","임석진 교수님","임황규 교수님"]

job = ["께서 소개해주신 인턴", "께서 추천해주신 부트캠프", "과 함께하는 논문작성", 
       "과 함께하는 학술대회", "의 학습튜터", "과 함께하는 연구", "께서 개최하시는 코딩테스트", 
       "의 직무특강", "이 추천하시는 자격증 준비", "과 함께하는 해외연수"]

plan = []

for loving in professor:
    for doing in job:
        plan.append(loving+doing)
        
# 1월 1일로부터 지난 날의 수를 받아, 달과 날로 반환하는 함수
def get_month_day_from_integer(day_number):
    start_date = datetime(2024, 1, 1)
    target_date = start_date + timedelta(days=day_number - 1)
    month = target_date.month
    day = target_date.day
    return month, day

# 달과 날을 받아, 1월 1일로부터 지난 날의 수를 반환하는 함수
def get_day_number(month, day):
    target_date = datetime(2024, month, day)
    start_date = datetime(2024, 1, 1)
    delta = target_date - start_date
    return delta.days + 1

In [ ]:
random.shuffle(plan)

# 랜덤 스케쥴 선택시 스케쥴의 시작, 종료날짜와 경험치, 성공확률을 랜덤으로 생성하는 함수
def random_schedule(N, v):
    random.seed()
    for i in range(N):
        s = random.randint(1, 299)
        e = random.randint(s, s+65)
        v[i] = (s,
                e,
                (e-s+1)*random.randint(800, 1200),
                random.randint(1, 99))

# 사용자 입력 스케쥴 선택시 사용자 정의 스케쥴 생성 함수
def input_schedule(N, v):
    print("***이벤트 입력(시작일 종료일 경험치 성공확률*1000)***")
    for i in range(N):
        print(f"\n{i+1}번째 이벤트 입력")
        while True:                              # 사용자가 잘못된 입력값을 넣을 경우 예외처리
            try:
                month, day = map(int, input("이벤트 시작일을 입력해주세요(ex: 1/19):").split("/"))
                start = get_day_number(month, day)
                month, day = map(int, input("이벤트 종료일을 입력해주세요(ex: 3/12):").split("/"))
                end = get_day_number(month, day)
            except:
                print("날짜를 형식에 맞추어 입력해주세요.")
                continue
            else:
                break
        
        while True:                              # 사용자가 잘못된 입력값을 넣을 경우 예외처리
            try:
                exp = int(input("이벤트를 통해 얻을 수 있는 경험치를 입력해주세요(1~100):"))
                pec = int(input("이벤트를 합격할 확률을 적어주세요(1~99):"))
            except:
                print("올바른 숫자를 입력해주세요.")
                continue
            else:
                break
                
        v[i] = (start, end, (end-start+1)*(exp+800), pec)

# 생성된 스케쥴의 기대치 최대화 함수
def top_down_dp_exvalue(n, end_time, v, schedule):
    if n == N:
        return (0, -1)
    if schedule[end_time][n][0] != 0:
        return schedule[end_time][n]
    if end_time > v[n][0]:
        schedule[end_time][n] = top_down_dp_exvalue(n+1, end_time, v, schedule)
        return schedule[end_time][n]
    a = top_down_dp_exvalue(n+1, end_time, v, schedule)
    b = top_down_dp_exvalue(n+1, v[n][1]+1, v, schedule)
    
    b = (b[0] + (v[n][2]*v[n][3]), n)
    schedule[end_time][n] = max(a, b)
    return schedule[end_time][n]

# 생성된 스케쥴의 경첨치 최대화 함수
def top_down_dp_value(n, end_time, v, schedule):
    if n == N:
        return (0, -1)
    if schedule[end_time][n][0] != 0:
        return schedule[end_time][n]
    if end_time > v[n][0]:
        schedule[end_time][n] = top_down_dp_value(n+1, end_time, v, schedule)
        return schedule[end_time][n]
    a = top_down_dp_value(n+1, end_time, v, schedule)
    b = top_down_dp_value(n+1, v[n][1]+1, v, schedule)
    b = (b[0] + v[n][2], n)
    schedule[end_time][n] = max(a, b)
    return schedule[end_time][n]

# 생성된 스케쥴의 이벤트 수 최대화 함수
def top_down_dp_maxcount(n, end_time, v, schedule):
    if n == N:
        return (0, -1)
    if schedule[end_time][n][0] != 0:
        return schedule[end_time][n]
    if end_time > v[n][0]:
        schedule[end_time][n] = top_down_dp_maxcount(n+1, end_time, v, schedule)
        return schedule[end_time][n]
    a = top_down_dp_maxcount(n+1, end_time, v, schedule)
    b = top_down_dp_maxcount(n+1, v[n][1]+1, v, schedule)
    b = (b[0] + 1, n)
    schedule[end_time][n] = max(a, b)
    return schedule[end_time][n]

while True:                              # 사용자가 잘못된 입력값을 넣을 경우 예외처리
    try:
        choice = int(input("랜덤으로 계획 생성하기 : 1\n사용자가 계획 입력하기 : 2\n"))
        if choice != 1 and choice != 2:
            raise
    except:
        print("1 혹은 2만 입력해주세요.")
        continue
    else:
        break
        
while True:                              # 사용자가 잘못된 입력값을 넣을 경우 예외처리
    try:
        N = int(input("이벤트 개수 입력"))
    except:
        print("올바른 개수를 입력해주세요.")
        continue
    else:
        break

v = [(0, 0, 0, 0) for _ in range(N)]

if choice == 1:
    random_schedule(N, v)
elif choice == 2:
    input_schedule(N, v)

schedule = [[(0, -1)] * N for _ in range(365)]

v.sort()
print()
print("올해 신청가능한 활동 리스트\n--------------------------------------------------")
for i in range(N):
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(plan[i]+"\n활동기간 : ", s_month, "/", s_day ," ~ ", e_month, "/", e_day,", 경험치 : ",v[i][2]," ,합격확률 : ",v[i][3],"%\n",sep="")

method = 1

s = top_down_dp_exvalue(0, 0, v, schedule)
i = s[1]

print(f"\n\n기대 경험치를 최대로 만들며 겹치지 않는 활동 리스트\n최대 기대 경험치: {s[0]//100}\n--------------------------------------------------")
idx = 1
while i != -1:
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(idx, end='. ')
    idx += 1
    print(plan[i]," (", s_month, "/", s_day, " ~ ", e_month, "/", e_day,")", sep="")
    day = v[i][1]
    if i==N-1:
        break
    i = schedule[day+1][i+1][1]

    
schedule = [[(0, -1)] * N for _ in range(365)]
s = top_down_dp_value(0, 0, v, schedule)
i = s[1]    
    
print(f"\n\n경험치를 최대로 만들며 겹치지 않는 활동 리스트\n최대 경험치: {s[0]}\n--------------------------------------------------")
idx = 1
while i != -1:
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(idx, end='. ')
    idx += 1
    print(plan[i]," (", s_month, "/", s_day, " ~ ", e_month, "/", e_day,")", sep="")
    day = v[i][1]
    if i==N-1:
        break
    i = schedule[day+1][i+1][1]
    
    
schedule = [[(0, -1)] * N for _ in range(365)]
s = top_down_dp_maxcount(0, 0, v, schedule)   
i = s[1]    

    
print(f"\n\n이벤트의 개수를 최대로 만들며 겹치지 않는 활동 리스트\n최대 이벤트 수: {s[0]}\n--------------------------------------------------")
idx = 1
while i != -1:
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(idx, end='. ')
    idx += 1
    print(plan[i]," (", s_month, "/", s_day, " ~ ", e_month, "/", e_day,")", sep="")
    day = v[i][1]
    if i==N-1:
        break
    i = schedule[day+1][i+1][1]
    